In [2]:
!uv add beautifulsoup4 pandas

               ▄▀▀▀▄                     
          ▄▀▀▀▄▀▀▀▀▀▀▀▀▄                 
         ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄▄              
     ▄▄  ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄     ▄▀▀▀▄  
   ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄  ▄▀▀▀▀▀▀▀▄
   ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀ ▀▀▀▀▀▀▀▀▀▀
     ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀ 
  ▄ ▄▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
 ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀     
  ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀     
 ▄▀▀▀▀▀▀▀ ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
▄▀▀▀▀▀▀  ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
▀▀▀▀▀▀ ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀   
      ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀    
       ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄         
      ▀▀▀▀▀▀▀▀   ▀▀▀▀  ▀▀▀▀▀▀▀▀▀         
                         ▀ ▀ ▀           

Resolved 130 packages in 8ms
Audited 126 packages in 11ms


In [3]:
import requests

url = "https://zenless-zone-zero.fandom.com/wiki/Agent/List"
res = requests.get(url)

In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(res.text, 'html.parser')

In [11]:
import pandas as pd

agents = []

playable_agents=soup.table.tbody.find_all('tr')[1:]
for agent_row in playable_agents:
    agent_tds=agent_row.find_all('td')
    agents.append({
        'icon':agent_tds[0].img.get('data-src') or agent_tds[0].img.get('src'),
        'name':agent_tds[1].a.text,
        'rank':agent_tds[2].span['title'].split(' ')[1].strip(),
        'attribute':agent_tds[3].a['title'],
        'speciality':agent_tds[4].a['title'],
        'attack_type':agent_tds[5].find('span',class_='zzw-icon-subtitle-text').text,
        'faction':agent_tds[6].a['title'],
        'release_date':list(agent_tds[7].stripped_strings)[0],
        'release_version':list(agent_tds[7].stripped_strings)[3].split(' ')[1],
    })

upcoming_agents=soup.find_all('table')[1].tbody.find_all('tr')[1:]
for agent_row in upcoming_agents:
    agent_tds=agent_row.find_all('td')
    agents.append({
        'icon':agent_tds[0].img.get('data-src') or agent_tds[0].img.get('src'),
        'name':agent_tds[1].a.text,
        'rank':None,
        'attribute':agent_tds[3].a['title'],
        'speciality':agent_tds[4].a['title'],
        'attack_type':None,
        'faction':agent_tds[6].a['title'],
        'release_date':None,
        'release_version':None,
    })
    
agents_df=pd.DataFrame(agents)
agents_df.head()

,icon,name,rank,attribute,speciality,attack_type,faction,release_date,release_version
0,https://static.wikia.nocookie.net/zenless-zone...,Alexandrina Sebastiane,S,Electric,Support,Strike,Victoria Housekeeping Co.,"July 04, 2024",1.0
1,https://static.wikia.nocookie.net/zenless-zone...,Alice Thymefield,S,Physical,Anomaly,Slash,Spook Shack,"August 06, 2025",2.1
2,https://static.wikia.nocookie.net/zenless-zone...,Anby Demara,A,Electric,Stun,Slash,Cunning Hares,"July 04, 2024",1.0
3,https://static.wikia.nocookie.net/zenless-zone...,Anton Ivanov,A,Electric,Attack,Pierce,Belobog Heavy Industries,"July 04, 2024",1.0
4,https://static.wikia.nocookie.net/zenless-zone...,Asaba Harumasa,S,Electric,Attack,Pierce,Hollow Special Operations Section 6,"December 18, 2024",1.4


In [12]:
import os

data_path='../data'
if not os.path.exists(data_path):
    os.mkdir(data_path)
agents_df.to_csv(f'{data_path}/all_agents.csv',index=False)